# INNER JOIN with Pandas

### 렉처 중간에 예제 문제가 있습니다. 풀면서 따라와주세요!

In [1]:
import pandas as pd

# 데이터 읽기

In [2]:
orders = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Orders.csv')
orderDetails = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/OrderDetails.csv')
products = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Customers.csv')
categories = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Categories.csv')
suppliers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Suppliers.csv')
shippers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Shippers.csv')
employees = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Employees.csv')

# 데이터 확인하기

In [6]:
print('columns in Orders: ' + str(list(orders.columns)))
print('columns in OrderDetails: ' + str(list(orderDetails.columns)))
print('columns in Products: ' + str(list(products.columns)))

columns in Orders: ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'ShipperID']
columns in OrderDetails: ['OrderDetailID', 'OrderID', 'ProductID', 'Quantity']
columns in Products: ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'Unit', 'Price']


In [7]:
print('총 ' + str(len(orders)) + '건의 주문 데이터')
print('총 ' + str(len(orderDetails)) + '건의 주문상세 데이터')

총 196건의 주문 데이터
총 518건의 주문상세 데이터


- 데이터 모델링 : 조인을 사용할 때 테이블간의 관계는 보통
- 1:1 / **1:n**  ex) 고객 한명당 여러개의 주문 / n:1/ m:n 이 있다.
- 1:1은 조인이 필요 없고, m:n은 현업에서 지양하는 부분(만나지 않길..!)

In [8]:
orders.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1


In [25]:
orderDetails.head(2)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10


In [26]:
products.head(2)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.0
1,2,Chang,1,1,24 - 12 oz bottles,19.0


# 분석의 목표
orders -> orderDetails -> products 를 묶어서 order 당 몇 개의 상품을 주문하는지, 종류는 얼마나 다양한지, 총 가격은 얼마인지 등을 구해보자.

## Order 한 개에 얼마나 다양한 물건들이 들어있는가?
Order 한 개 당 몇 개의 ProductID를 가지고 있는지 세보라.

In [10]:
'''
SELECT *
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
'''

order_orderDetails = orders.merge(orderDetails, how='inner', on='OrderID')
# 양쪽 칼럼 이름이 같으면 On 이고, 다르면 left_on 이나 right_on 으로 쓰면 된다. (shft+tap 해서 규칙보기)
# left_index = False -> left index는 안 쓰겠다 라는 뜻 쓰고싶으면 True 로 하면 됨 
# - 데이터에서 =이 없는 얘들은 우리가 값을 입력해야하고, =이 있으면 Defalt 값이당
#     - ex) by 는 입력해야하고, ascending=True는 기본값

In [12]:
order_orderDetails.head(3)
# orderid 10248에 orderdetailld가 1,2,3 개가 붙음

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity
0,10248,90,5,1996-07-04,3,1,11,12
1,10248,90,5,1996-07-04,3,2,42,10
2,10248,90,5,1996-07-04,3,3,72,5


In [13]:
'''
SELECT o.OrderID, COUNT(*)
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.orderID = od.OrderID
GROUP BY o.OrderID
'''
# order_orderDetails : 아까 innder join 해준 테이블 
products_per_order = order_orderDetails.groupby('OrderID').size()
products_per_order

OrderID
10248    3
10249    2
10250    3
10251    3
10252    3
        ..
10439    4
10440    4
10441    1
10442    3
10443    2
Length: 196, dtype: int64

In [14]:
# 평균적으로 1오더 당 3가지의 Product가 들어있다.
products_per_order.mean()

2.642857142857143

## Order 한 개에 총 몇 개의 물건이 들어있는가?

In [56]:
'''
SELECT o.OrderID
     , SUM(od.Quantity)
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
GROUP BY o.OrderID
'''

num_items_per_order = order_orderDetails.groupby('OrderID')['Quantity'].sum()
num_items_per_order

OrderID
10248     27
10249     49
10250     60
10251     41
10252    105
        ... 
10439     67
10440    208
10441     50
10442    170
10443     18
Name: Quantity, Length: 196, dtype: int64

In [38]:
# 한 주문 당 평균적으로 65.02개의 물건이 들어있다.
num_items_per_order.mean()

65.01530612244898

# Q. 65개 이상의 물건이 들어있는 주문의 OrderID를 구하세요. 

In [17]:
# Orders , OrderDetails
order_join = orders.merge(orderDetails, how='inner', on ='OrderID')

In [20]:
order_sum =order_join.groupby('OrderID')['Quantity'].sum()

In [42]:
order_sum_data=order_sum.reset_index() #pandas_array를 index로 reset 시키면 dataframe이 됨 그리고 한번 더 테이블 이름 지정 

In [44]:
order_sum_data[order_sum_data['Quantity'] >= 65] #테이블을 >= 65 하면 true,false로 나오니까 한번 더 감싸주기 

,OrderID,Quantity
4,10252,105
5,10253,102
7,10255,110
10,10258,121
12,10260,102
15,10263,184
19,10267,135
21,10269,80
24,10272,70
25,10273,152


## 주문마다 총 가격을 구해주세요.

In [45]:
# SQL로는 이렇게 쓸 수 있겠죠!

'''
SELECT od.OrderID
     , SUM(od.Quantity*p.Price)
FROM OrderDetails AS od
     INNER JOIN Products AS p ON od.ProductID = p.ProductID
GROUP BY od.OrderID
'''

'\nSELECT od.OrderID\n     , SUM(od.Quantity*p.Price)\nFROM OrderDetails AS od\n     INNER JOIN Products AS p ON od.ProductID = p.ProductID\nGROUP BY od.OrderID\n'

In [51]:
# 1. orderdetials, prodcts merge
# 2. qunatity * price 곱한 필드를 만들기 
# 3. orderid 별로 groupby sum() 해주세요 
order_product=order_join.merge(products, how = 'inner', on = 'ProductID')

In [72]:
order_product['total'] =order_product['Price'] * order_product['Quantity']

In [82]:
big = order_product.groupby('OrderID')['total'].sum()

In [78]:
# 가장 큰 값별로 저장 

In [86]:
big.sort_values(ascending=False) #desc 해주기 


OrderID
10372    15353.60
10424    14366.50
10417    14104.00
10353    13427.00
10360     9244.25
           ...   
10308      111.00
10281      108.20
10391      108.00
10422       62.46
10271       60.00
Name: total, Length: 196, dtype: float64

In [91]:
# 만약 reset_index()하고 해주고 싶다면 by를 추가하면 된다
big.reset_index().sort_values(by='total', ascending=False)

,OrderID,total
124,10372,15353.60
176,10424,14366.50
169,10417,14104.00
105,10353,13427.00
112,10360,9244.25
76,10324,7698.45
192,10440,7246.01
182,10430,7245.00
103,10351,7103.60
81,10329,6025.12


In [41]:
# 선미님의 풀이 
# 변수명 이렇게 하시면 나중에 헷갈리고 힘들어요... 저는 강의니까 그냥 이렇게 하께요 (변명)
order_all = order_orderDetails.merge(products, how='inner', on='ProductID')
order_all.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
0,10248,90,5,1996-07-04,3,1,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
1,10296,46,6,1996-09-03,1,130,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0


In [70]:
# 컬럼이 너무 많으니까 일단 지금 관심 있는 것만 추려볼게요
order_all_part = order_all[['OrderID', 'OrderDate', 'Quantity', 'ProductID', 'CategoryID', 'Price']]

In [73]:
total_price = order_all_part['Price'] * order_all_part['Quantity']
order_all_part['TotalPricePerProduct'] = total_price

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
price_per_order = order_all_part.groupby('OrderID')['TotalPricePerProduct'].sum()
price_per_order

OrderID
10248     566.00
10249    2329.25
10250    2267.25
10251     839.50
10252    4662.50
          ...   
10439    1348.70
10440    7246.01
10441    2195.00
10442    2246.00
10443     673.20
Name: TotalPricePerProduct, Length: 196, dtype: float64

## 총 가격이 1000불 이상인 주문만 뽑아주세요.

In [77]:
# SQL로는 이렇게 쓸 수 있겠죠!

'''
SELECT o.OrderID
     , SUM(od.Quantity*p.Price) AS PricePerOrder
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
     INNER JOIN Products AS p ON od.ProductID = p.ProductID
GROUP BY o.OrderID
HAVING PricePerOrder >= 1000
'''

'\nSELECT o.OrderID\n     , SUM(od.Quantity*p.Price) AS PricePerOrder\nFROM Orders AS o\n     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID\n     INNER JOIN Products AS p ON od.ProductID = p.ProductID\nGROUP BY o.OrderID\nHAVING PricePerOrder >= 1000\n'

In [82]:
price_per_order[price_per_order >= 1000]

OrderID
10249    2329.25
10250    2267.25
10252    4662.50
10253    1806.00
10255    3115.75
          ...   
10436    2763.50
10439    1348.70
10440    7246.01
10441    2195.00
10442    2246.00
Name: TotalPricePerProduct, Length: 118, dtype: float64

### Q1. Products의 CategoryID와 Categories 테이블을 이용해서, 어떤 카테고리의 상품이 가장 많이 주문되었는지 구하세요.

In [84]:
# 풀이

### Q2. Orders 테이블의 OrderDate를 이용해, 어떤 날에 주문이 가장 많았는지 알려주세요.

In [85]:
# 풀이

### Q3. Orders 테이블의 OrderDate를 이용해, 어떤 날에 매출금액이 가장 높았는지 알려주세요.

In [86]:
# 풀이

### Q4 문제를 만들고, 풀어보세요!